In [1]:
import pandas as pd
from pandas import DataFrame
import os

In [ ]:
# filepath = './data/netflix_titles.csv'

# df = pd.read_csv(filepath, header=0, sep=',').replace('\n','', regex=True)

def csv_to_dataframe(filepath:str, index:str) -> None:
    """
    ### Load a csv file into a pandas dataframe.

    Returns inserted csv into 'df' variable.

     - filepath: string of filepath to csv file.

     - index: string of column to index dataframe by.
    """
    df = pd.read_csv(filepath, header=0, index_col=[index])
    return df

In [41]:
# Dataframe Details
df = pd.read_csv("https://media.geeksforgeeks.org/wp-content/uploads/nba.csv")
def details(df:DataFrame):
    """
    Prints Stats of a dataframe, Returning columns, the shape and datatypes of the columns
    """
    print(f"""
    Columns: {list(df.columns)}
    Shape: {df.shape[0]} Rows, {df.shape[1]} Columns
    Data Types: {list(df.dtypes)}
    """)



In [ ]:
def index_setter(df:DataFrame, index) -> None:
    """
    Sets the index of dataframe to a desired existing column
        - df: dataframe to have index changed
        - index: name of column that index will be set to
    """
    df.set_index(index, inplace=True)

In [ ]:


def remove_new_line_char(df:DataFrame) -> DataFrame:
    """
    Removes new lines from dataframe.
    """
    return df.replace('\n','', regex=True)

In [ ]:
def nullfill(df:DataFrame, fill_val) -> None:
    """
    Targets empty fields of a dataframe and sets them to the fill_val
    """    
    df.fillna(value=fill_val, axis=1, inplace=True)

In [ ]:
def duplicate_drop(df:DataFrame, col_list=None) -> None:
    """
    drop duplicate rows targeting a specific column(s)
    col_list: ['column1', 'column2']
    - Enter column name as a string
    - Only 1 column required, if none set uses all columns
    - Enter multiple columns as strings seperated by commas in a list
    - eg: duplicate drop(dataframe, ['col1', 'col2'])
    """
    df.drop_duplicates(subset=col_list, inplace=True)

In [ ]:
def drop_missing_row(df:DataFrame, col_list=None) -> None:
    """
    Drops dataframe row if it is missing any data in col_list
    col_list: ['column1', 'column2']
    - Enter column name as a string or list
    - Only 1 column required, if none set uses all columns
    - Enter multiple columns as strings seperated by commas in a list
    - eg: drop_missing_row(dataframe, ['col1', 'col2'])
    """
    df.dropna(subset=col_list, inplace=True)

In [ ]:
def drop_low_data(df:DataFrame, threshold=2) -> None:
    """
    Drops dataframe row if missing more than x(threshold) values, Default is 2 
    """
    df.dropna(thresh=threshold, inplace=True)

In [42]:
def drop_extra_columns(df:DataFrame, num_cols):
    """
    Drops dataframe columns if dataframe has more than <num_cols> columns
    """
    df.drop(df.columns[num_cols:])

In [46]:

def write_csv(df:DataFrame, filepath, filename):
    """
    Writes a dataframe into a new csv file located in filepath
    """
    os.makedirs(filepath, exist_ok=True)  
    df.to_csv(f'{filepath}{filename}')

In [ ]:
export DISPLAY=`grep -oP "(?<=nameserver ).+" /etc/resolv.conf`:0.0

In [1]:
import PySimpleGUI as sg

# --------- SAMPLE GUI ----------- Not my code. PySimpleGUI

sg.theme('DarkAmber')   # Add a touch of color
# All the stuff inside your window.
layout = [  [sg.Text('Some text on Row 1')],
            [sg.Text('Enter something on Row 2'), sg.InputText()],
            [sg.Button('Ok'), sg.Button('Cancel')] ]

# Create the Window
window = sg.Window('Window Title', layout)
# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break
    print('You entered ', values[0])

window.close()

You entered  2


In [2]:
import tkinter as tk

window = tk.Tk()
label = tk.Label(text="Test Tkinter!", fg="white", bg="black", height=10, width=10)
label.pack()
button = tk.Button(
    text="Click me!",
    width=25,
    height=5,
    bg="blue",
    fg="yellow",
)
button.pack()
entry = tk.Entry(fg="yellow", bg="blue", width=50)
entry.pack()

window.mainloop()


In [ ]:
import tkinter as tk


border_effects = {

    "flat": tk.FLAT,

    "sunken": tk.SUNKEN,

    "raised": tk.RAISED,

    "groove": tk.GROOVE,

    "ridge": tk.RIDGE,

}


window = tk.Tk()


for relief_name, relief in border_effects.items():

    frame = tk.Frame(master=window, relief=relief, borderwidth=5)

    frame.pack(side=tk.LEFT)

    label = tk.Label(master=frame, text=relief_name)

    label.pack()


window.mainloop()

In [35]:
from tkinter import *
from tkinter.ttk import *
from tkinter.filedialog import askopenfile
import time

ws = Tk()
ws.title('CSV Cleaner')
ws.geometry('400x200')

def open_file():
    file_path = askopenfile(mode='r', filetypes=[('CSV Files', '*.csv')])
    if file_path is not None:
        label.configure(text=file_path)


def uploadFiles():
    pb1 = Progressbar(
        ws, 
        orient=HORIZONTAL, 
        length=300, 
        mode='determinate',
        bootstyle="success"
        )
    pb1.grid(row=4, columnspan=3, pady=20)
    for i in range(5):
        ws.update_idletasks()
        pb1['value'] += 20
        time.sleep(1)
    pb1.destroy()
    Label(ws, text='File Uploaded Successfully!', foreground='green').grid(row=4, columnspan=3, pady=10)
    
# label=Label(ws, text="", font=("Courier 22 bold"))
# label.pack()    
    
    
adhar = Label(
    ws, 
    text='Upload CSV to be cleaned '
    )
adhar.grid(row=0, column=0, padx=10)

adharbtn = Button(
    ws, 
    text ='Choose File', 
    command = lambda:open_file()
    ) 
adharbtn.grid(row=0, column=1)

upld = Button(
    ws, 
    text='Upload Files', 
    command=uploadFiles
    )
upld.grid(row=3, columnspan=3, pady=10)



ws.mainloop()

In [33]:
#Import the required Libraries
from tkinter import *
from tkinter import ttk

#Create an instance of Tkinter frame
win= Tk()

#Set the geometry of Tkinter frame
win.geometry("750x250")

def display_text():
   global entry
   string= entry.get()
   label.configure(text=string)

#Initialize a Label to display the User Input
label=Label(win, text="", font=("Courier 22 bold"))
label.pack()

#Create an Entry widget to accept User Input
entry= Entry(win, width= 40)
entry.focus_set()
entry.pack()

#Create a Button to validate Entry Widget
ttk.Button(win, text= "Okay",width= 20, command= display_text).pack(pady=20)

win.mainloop()

In [45]:
# Import the required Libraries
import tkinter as tk
from tkinter import ttk, filedialog
from tkinter import *
from tkinter.filedialog import askopenfile
import ttkbootstrap as ttk
import os
from ttkbootstrap.constants import *
import pandas as pd
from pandas import DataFrame


# Create an instance of tkinter frame
# win = Tk()
win = ttk.Window(themename="solar")
win.title("Dylan's Utility App")
# Set the theme

# Set the geometry of tkinter frame
win.geometry("950x500")

def open_file():
   file = filedialog.askopenfile(mode='r', filetypes=[('CSV File', '*.csv')])
   if file:
      global filepath
      filepath = os.path.abspath(file.name)
      # Label(win, text="The File is located at : " + str(filepath)).pack()
      label2.config(text="The File is located at : " + str(filepath))

def csv_to_dataframe(filepath:str) -> None:
   """
   ### Load a csv file into a pandas dataframe.

   Returns inserted csv into 'df' variable.

   - filepath: string of filepath to csv file.
   """
   global df
   df = pd.read_csv(filepath, header=0)
   

def details(df:DataFrame):
   """
   Prints Stats of a dataframe, Returning columns, the shape and datatypes of the columns
   """
   text1.insert('end',
   f"""
   Columns: {list(df.columns)}
   Shape: {df.shape[0]} Rows, {df.shape[1]} Columns
   Data Types: {list(df.dtypes)}
   """)
   text1.config(state='disabled', yscrollcommand=sb.set)
   


# Add a Label widget
label1 = Label(win, text="Click the Button to browse the Files")
label1.pack(pady=10)

# File browser, returns filepath
ttk.Button(win,
   text="Browse", 
   command=open_file, 
   bootstyle=PRIMARY)\
   .pack(pady=10)

label2 = Label(win, text="The File is located at : ")
label2.pack(pady=10)
# Create dataframe in pandas for transformations
ttk.Button(win,
   text="Create Pandas Dataframe",
   command=lambda: csv_to_dataframe(filepath),
   bootstyle=PRIMARY)\
   .pack(pady=10)

# list details of df
ttk.Button(win,
   text="Display Details of Dataframe: ",
   command=lambda: details(df),
   bootstyle=PRIMARY)\
   .pack(pady=10)

# label3 = Label(win, text="")
# label3.pack(pady=5)
frame = Frame(win)
text1 = Text(frame, height = 5, width = 80, wrap='word')
text1.pack(side =LEFT, expand=True)
sb = Scrollbar(frame)
sb.pack(side=RIGHT, fill=BOTH)
sb.config(command=text1.yview)
frame.pack(expand=True)




win.mainloop()

bgerror failed to handle background error.
    Original error: can't invoke "event" command: application has been destroyed
    Error in bgerror: can't invoke "tk" command: application has been destroyed
